In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
from pprint import pprint

# Resource File (CSV)
five_cities = pd.read_csv("output_data/restaurants_category_10_five_cities.csv")
df = five_cities

In [30]:
#drop null values
df.sort_values("Average Cost for two")

Unnamed: 0  Restaurant_id                                     Name  \
448          48       16587373  Cam Wah Chinese & Vietnamese Restaurant   
303           3       16598976                                   Marumo   
321          21       16600545                           Bossman Coffee   
58           58       16568870                       Lentil As Anything   
254          54       16591575                            Just Soy Cafe   
..          ...            ...                                      ...   
162          62       18235287                                    Amaru   
1             1       16558798                                     Quay   
110          10       16570775                                   Attica   
102           2       16572612                             Vue de monde   
414          14       16608070             d'Arry's Verandah Restaurant   

            Locality                                            Address  \
448         Kilkenny  33 Kilkenny Rd, Kilkenny Park, Kilkenny, Adelaide   
303         Nedlands  22/145 Stirling Hwy, Nedlands, Nedlands & Dalk...   
321     Mount Lawley         3/669 Beaufort Street, Mount Lawley, Perth   
58           Newtown                   391 King Street, Newtown, Sydney   
254  Sunnybank Hills  Pacific Centre, Shop 1, 8 Lear Street, Sunnyba...   
..               ...                                                ...   
162         Armadale      Shop 5, 1121 High Street, Armadale, Melbourne   
1      Circular Quay  Upper Level, Overseas Passenger Terminal 5 Hic...   
110        Ripponlea            74 Glen Eira Road, Ripponlea, Melbourne   
102              CBD  Level 55, Rialto, 525 Collins Street, CBD, Mel...   
414     McLaren Vale                            Osborn Rd, McLaren Vale   

          City   Latitude   Longitude  Zip Code  \
448   Adelaide -34.886585  138.548981    5011.0   
303      Perth -31.980216  115.796659    6907.0   
321      Perth -31.933771  115.872299    6050.0   
58      Sydney -33.899735  151.177852    2042.0   
254   Brisbane -27.610180  153.054240    4109.0   
..         ...        ...         ...       ...   
162  Melbourne -37.855972  145.024943       NaN   
1       Sydney -33.858029  151.209970    2000.0   
110  Melbourne -37.876924  144.997312    3185.0   
102  Melbourne -37.818954  144.957934    3000.0   
414   Adelaide -35.198372  138.545242    5171.0   

                                 Cuisines  Price Range  Average Cost for two  \
448                   Chinese, Vietnamese            1                     0   
303  Japanese, Seafood, Modern Australian            1                     0   
321                             Cafe Food            1                    15   
58                      Vegetarian, Vegan            1                    20   
254                       Desserts, Asian            1                    20   
..                                    ...          ...                   ...   
162                          Contemporary            4                   450   
1                       Modern Australian            4                   500   
110              Australian, Contemporary            4                   600   
102              Australian, Contemporary            4                   600   
414         Modern Australian, Australian            1              25000017   

     User Rating Rating Text  Votes  all_reviews_count  
448          4.4   Very Good     97                 29  
303          4.9   Excellent    437                140  
321          4.6   Excellent    297                105  
58           4.6   Excellent    195                 92  
254          4.5   Excellent    237                 76  
..           ...         ...    ...                ...  
162          4.6   Excellent    331                126  
1            4.9   Excellent   1366                454  
110          4.8   Excellent   1485                491  
102          4.9   Excellent   3225                986  
41

In [31]:
#drop values where average cost for two is not valid - either $0 or 25000017
clean_cost_for2 = df.loc[(df["Average Cost for two"] != 0) &
                        (df["Average Cost for two"] != 25000017)]
clean_cost_for2["Average Cost for two"].mean()

90.6036217303823

In [36]:
#sort dataframe by user rating and check the data
clean_df = clean_cost_for2
clean_df.sort_values("User Rating")

Unnamed: 0  Restaurant_id                        Name  \
499          99       16586902             East Taste Cafe   
498          98       16589555            Sukhumvit Soi 38   
484          84       18050804                Osteria Oggi   
486          86       16588258                 Cafe Troppo   
487          87       16586941                    Gaucho's   
..          ...            ...                         ...   
104           4       16574138                     Suzuran   
105           5       16575773  Scopri Italian Food & Wine   
106           6       16575846   Pho Hung Vuong Springvale   
200           0       16593535                Rogue Bistro   
0             0       15547004           Restaurant Hubert   

                         Locality  \
499    Gouger Street, City Centre   
498                   City Centre   
484     Pirie Street, City Centre   
486  Whitmore Square, City Centre   
487    Moonta Street, City Centre   
..                            ...   
104                    Camberwell   
105                       Carlton   
106                    Springvale   
200                      Newstead   
0                             CBD   

                                               Address       City   Latitude  \
499                        119 Gouger Street, Adelaide   Adelaide -34.930500   
498                       54 Pulteney Street, Adelaide   Adelaide -34.923131   
484             76 Pirie Street, City Centre, Adelaide   Adelaide -34.925750   
486                       42 Whitmore Square, Adelaide   Adelaide -34.933199   
487                         91 Gouger Street, Adelaide   Adelaide -34.930364   
..                                                 ...        ...        ...   
104             1025 Burke Road, Camberwell, Melbourne  Melbourne -37.821765   
105          191, Nicholson Street, Carlton, Melbourne  Melbourne -37.795718   
106  Shop 2, 15 Balmoral Avenue, Springvale, Melbourne  Melbourne -37.949409   
200               14 Austin Street, Newstead, Brisbane   Brisbane -27.445894   
0                         15 Bligh Street, CBD, Sydney     Sydney -33.865348   

      Longitude  Zip Code                   Cuisines  Price Range  \
499  138.595184    5000.0             Asian, Chinese            2   
498  138.605479    5000.0                       Thai            3   
484  138.602556    5000.0          European, Italian            3   
486  138.594901    5000.0  Coffee and Tea, Cafe Food            2   
487  138.595978    5000.0           Argentine, Steak            4   
..          ...       ...                        ...          ...   
104  145.058465    3124.0            Japanese, Sushi            2   
105  144.975115    3053.0                    Italian            4   
106  145.151019    3171.0            Vietnamese, Pho            2   
200  153.044373    4006.0          Modern Australian            4   
0    151.210624    2000.0           French, European            4   

     Average Cost for two  User Rating Rating Text  Votes  all_reviews_count  
499                    50          4.2   Very Good    260                 82  
498                    90          4.2   Very Good    145                 78  
484                    95          4.2   Very Good    254                116  
486                    40          4.2   Very Good    299                111  
487                   160          4.2   Very Good    505                171  
..                    ...          ...         ...    ...                ...  
104                    35          4.9   Excellent    803                194  
105                   180          4.9   Excellent    458                193  
106                    40          4.9   Excellent    923                305  
200                   140          4.9   Excellent    491                205  
0                     150          4.9   Excellent    589                224  

[497 rows x 16 columns]

In [42]:
#sort dataframe by number of votes and check the data
clean_df.sort_values(by="Votes", ascending= False)
clean_df

Unnamed: 0  Restaurant_id                    Name  \
0             0       15547004       Restaurant Hubert   
1             1       16558798                    Quay   
2             2       16559171               Tetsuya's   
3             3       16569454       LuMi Bar & Dining   
4             4       15545439                 Manpuku   
..          ...            ...                     ...   
495          95       16586992      Jolley's Boathouse   
496          96       16588907             The Gallery   
497          97       16587014  Lenzerheide Restaurant   
498          98       16589555        Sukhumvit Soi 38   
499          99       16586902         East Taste Cafe   

                         Locality  \
0                             CBD   
1                   Circular Quay   
2                             CBD   
3                         Pyrmont   
4                       Chatswood   
..                            ...   
495                   City Centre   
496  Waymouth Street, City Centre   
497                     Kingswood   
498                   City Centre   
499    Gouger Street, City Centre   

                                               Address      City   Latitude  \
0                         15 Bligh Street, CBD, Sydney    Sydney -33.865348   
1    Upper Level, Overseas Passenger Terminal 5 Hic...    Sydney -33.858029   
2                         529 Kent Street, CBD, Sydney    Sydney -33.875143   
3          56 Pirrama Road, \tPyrmont, Pyrmont, Sydney    Sydney -33.867137   
4               226 Victoria Avenue, Chatswood, Sydney    Sydney -33.794417   
..                                                 ...       ...        ...   
495              1 Jolleys Lane, City Centre, Adelaide  Adelaide -34.917011   
496  30 Waymouth Street, Adelaide, City Centre, Ade...  Adelaide -34.925872   
497               146 Belair Road, Kingswood, Adelaide  Adelaide -34.972047   
498                       54 Pulteney Street, Adelaide  Adelaide -34.923131   
499                        119 Gouger Street, Adelaide  Adelaide -34.930500   

      Longitude  Zip Code                    Cuisines  Price Range  \
0    151.210624    2000.0            French, European            4   
1    151.209970    2000.0           Modern Australian            4   
2    151.204932       NaN                    Japanese            4   
3    151.197517    2009.0           Italian, Japanese            4   
4    151.189542    2067.0             Japanese, Ramen            2   
..          ...       ...                         ...          ...   
495  138.600296    5000.0  Seafood, Modern Australian            4   
496  138.598311    5000.0             Tapas, Bar Food            4   
497  138.609192    5062.0                    European            4   
498  138.605479    5000.0                        Thai            3   
499  138.595184    5000.0              Asian, Chinese            2   

     Average Cost for two  User Rating Rating Text  Votes  all_reviews_count  
0                     150          4.9   Excellent    589                224  
1                     500          4.9   Excellent   1366                454  
2                     440          4.9   Excellent   1234                329  
3                     190          4.9   Excellent    452                196  
4                      40          4.9   Excellent    486                190  
..                    ...          ...         ...    ...                ...  
495                   120          4.2   Very Good    233                 93  
496                   100          4.2   Very Good    166                 89  
497                   120          4.2   Very Good    233                 82  
498                    90          4.2   Very Good    145                 78  
499                    50          4.2   Very Good    260                 82  

[497 rows x 16 columns]

In [48]:
#Reorganising columns so Cuisine is at the end of the dataframe - dropping unnamed.
new_df = clean_df[['Restaurant_id',"Name", "Locality", "Address", "City", "Latitude", "Longitude", "Zip Code", "Price Range", 
                      "Average Cost for two", "User Rating", "Rating Text", "Votes", "all_reviews_count", "Cuisines"]]


In [49]:
#Separating Cuisines data by comma and placing the data into new columns
df = pd.concat([new_df, new_df['Cuisines'].str.split(', ', expand=True)], axis=1)

In [50]:
df.count()

Restaurant_id           497
Name                    497
Locality                497
Address                 497
City                    497
Latitude                497
Longitude               497
Zip Code                478
Price Range             497
Average Cost for two    497
User Rating             497
Rating Text             497
Votes                   497
all_reviews_count       497
Cuisines                497
0                       497
1                       310
2                        76
3                        14
4                         4
5                         2
6                         2
7                         2
dtype: int64

In [51]:
df.to_csv("output_data/clean_fiveCities.csv")